In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
page =requests.get('https://www.carwale.com/tata-cars/')

In [5]:
soup = BeautifulSoup(page.content,'html.parser')

In [20]:
type(soup)

bs4.BeautifulSoup

In [17]:
test=soup.find('ul',class_='content-box-shadow')

In [22]:
test2 =test.find_all('li',class_='list-seperator')

In [23]:
len(test2)

15

In [24]:
test2[0]

<li class="list-seperator">
<div class="grid-5 alpha">
<div class="just-launched-section"><div class="launch-title">
LAUNCHED ON							</div><div class="launch-date-unit"><span class="launch-date">23<span class="lauch-date-unit">rd</span></span><br/><span class="launch-month">JAN</span></div></div>
<a href="/tata-cars/harrier/"><img alt="Tata Harrier" border="0" class="lazy" data-action="Image-click" data-cat="MakePage" data-event="CWInteractive" data-label="Harrier" data-original="https://imgd.aeplcdn.com/227x128/cw/ec/32958/Tata-Harrier-Right-Front-Three-Quarter-142878.jpg?wm=0&amp;q=85" data-role="click-tracking" height="128" src="https://imgd.aeplcdn.com/227X128/cw/cars/no-cars.jpg" title="Harrier" width="227"/></a></div><div class="grid-7 omega"><a class="font18" data-action="Title-click" data-cat="MakePage" data-event="CWInteractive" data-label="Harrier" data-role="click-tracking" href="/tata-cars/harrier/" title="Tata Harrier"><strong class="text-unbold">Tata Harrier</strong></a

In [34]:
test2[0].find('div',class_='grid-7 omega').a.get('title')

'Tata Harrier'

In [ ]:
https://www.carwale.com/tata-cars/harrier/userreviews/

In [192]:
new_url ='https://www.carwale.com'+test2[0].find('div',class_='grid-7 omega').a.get('href')+'userreviews'

In [193]:
new_url

'https://www.carwale.com/tata-cars/harrier/userreviews'

In [43]:
innerPage = requests.get(new_url)

soup2 = BeautifulSoup(innerPage.content,'html.parser')

In [47]:
all_reviews =soup2.find('div',id='userReviewListing').find_all('div')

In [51]:
review_page ='https://www.carwale.com'+all_reviews[0].find('a').get('href')

In [55]:
review_page

'https://www.carwale.com/tata-cars/harrier/userreviews/74223/'

In [52]:
review_contents =requests.get(review_page)

In [53]:
soup3 = BeautifulSoup(review_contents.content,'html.parser')

In [69]:
txt1 =soup3.h4.text.replace('\r','').replace('\n','').replace('\t','')

txt1

'A user review on Tata Harrier. Written byTUSHARKUMAR PARMAR on 25 January, 2019'

In [74]:
user_name =re.search('by(.+)on(.+)',txt1).group(1)

user_name

'TUSHARKUMAR PARMAR '

In [72]:
re.search('by(.+)on(.+)',txt1).group(2)

' 25 January, 2019'

In [76]:
review_date =pd.to_datetime(re.search('by(.+)on(.+)',txt1).group(2))

review_date

Timestamp('2019-01-25 00:00:00')

In [119]:
overall =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[0].span.find_all('img'))))

overall

5

In [127]:
looks =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[1].span.find_all('img'))))

looks

5

In [128]:
performance =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[2].span.find_all('img'))))

performance


5

In [130]:
comfort =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[3].span.find_all('img'))))

comfort


5

In [133]:
economy =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[4].span.find_all('img'))))

economy

4

In [134]:
value_for_money =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[5].span.find_all('img'))))

value_for_money

5

In [163]:
Version =soup3.find('tr',id='trVerReviewed').text.replace('\n','').split(':')[1]

Version

'XZ'

In [145]:
review_liked =int(soup3.find('span',id='spnLiked').text)

review_liked

2

In [146]:
total_people_liked =int(soup3.find('span',id='spnDisliked').text)

total_people_liked

2

In [152]:
full_review_text =soup3.find_all('div',class_='mid-box margin-top20')[3].div.text

full_review_text

'Good looking and very good performance. This car is more batter then other. Exterior and interior design is so good for other car. \n\nCar is more spece  to other car. Tata is making  good vehicles  in last few years.'

In [161]:
purchase_condition =str(re.search(':(.+)Familiarity',soup3.find_all('div',class_='mid-box margin-top20')[2].text.replace('\n','')).group(1))
purchase_condition

'New'

In [165]:
familiarity = str(re.search('Familiarity:(.+)',soup3.find_all('div',class_='mid-box margin-top20')[2].text.replace('\n','')).group(1))
familiarity

'Have driven for a few hundred kilometres'

### Creating a function to extract out of a single review page

In [225]:
def get_review_content(review_page):
    content={}  
    
    review_contents =requests.get(review_page)
    
    soup3 = BeautifulSoup(review_contents.content,'html.parser')
    
# Get username and review date details
    txt1 =soup3.h4.text.replace('\r','').replace('\n','').replace('\t','')
    if re.search('by(.+)on(.+)',txt1)!=None:
        user_name =re.search('by(.+)on(.+)',txt1).group(1)
    else:
        user_name='User'
    
    if re.search('by(.+)on(.+)',txt1)!=None:
        review_date =pd.to_datetime(re.search('by(.+)on(.+)',txt1).group(2))
    else:
        review_date=pd.to_datetime('1900-01-01')
        
    content['user_name'] = user_name
    content['review_date'] = review_date
    
#Get users general review
    overall =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[0].span.find_all('img'))))
    looks =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[1].span.find_all('img'))))
    performance =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[2].span.find_all('img'))))
    comfort =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[3].span.find_all('img'))))
    economy =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[4].span.find_all('img'))))
    value_for_money =sum(list(map(lambda x :1 if re.search('1.png',x.get('src'))!=None else 0,soup3.find('ul',class_='model-ratings vr-dotted').find_all('li')[5].span.find_all('img'))))
    content['overall'] = overall
    content['looks'] = looks
    content['performance'] = performance
    content['comfort'] = comfort
    content['economy'] = economy
    content['value_for_money'] = value_for_money
    
#Model of the Car reviewed
    if soup3.find('tr',id='trVerReviewed')!=None:
        model =soup3.find('tr',id='trVerReviewed').text.replace('\n','').split(':')[1]
    else:
        model='Not Mentioned'
    content['model'] =model

#Purchase Condition and Familiarity

    if re.search(':(.+)Familiarity',soup3.find_all('div',class_='mid-box margin-top20')[2].text.replace('\n',''))!=None:
        purchase_condition =str(re.search(':(.+)Familiarity',soup3.find_all('div',class_='mid-box margin-top20')[2].text.replace('\n','')).group(1))
    else:
        purchase_condition='Not Mentioned'
    
    if re.search('Familiarity:(.+)',soup3.find_all('div',class_='mid-box margin-top20')[2].text.replace('\n',''))!=None:
        familiarity = str(re.search('Familiarity:(.+)',soup3.find_all('div',class_='mid-box margin-top20')[2].text.replace('\n','')).group(1))
    else:
        familiarity='Not Mentioned'
    
    content['purchase_condition'] =purchase_condition
    content['familiarity'] =familiarity
    
#Full Review

    if soup3.find_all('div',class_='mid-box margin-top20')[3].div!=None:
        full_review_text =soup3.find_all('div',class_='mid-box margin-top20')[3].div.text
    else:
        full_review_text=""
        content['Review'] = full_review_text

#Helpfulness of the review

    if soup3.find('span',id='spnLiked')!=None:
        review_liked =int(soup3.find('span',id='spnLiked').text)
    else:
        review_liked=0
    
    if soup3.find('span',id='spnDisliked')!=None:
        total_people_liked =int(soup3.find('span',id='spnDisliked').text)
    else:
        total_people_liked=0
    
    content['like'] = review_liked
    content['total_likes'] = total_people_liked
    
    return(content)   

    

In [168]:
get_review_content(review_page)

{'Review': 'Good looking and very good performance. This car is more batter then other. Exterior and interior design is so good for other car. \n\nCar is more spece  to other car. Tata is making  good vehicles  in last few years.',
 'comfort': 5,
 'economy': 4,
 'familiarity': 'Have driven for a few hundred kilometres',
 'like': 2,
 'looks': 5,
 'model': 'XZ',
 'overall': 5,
 'performance': 5,
 'purchase_condition': 'New',
 'review_date': Timestamp('2019-01-25 00:00:00'),
 'total_likes': 2,
 'user_name': 'TUSHARKUMAR PARMAR ',
 'value_for_money': 5}

### Get all Reviews for the same car

In [202]:
def get_car_reviews(url):
    import math
    url_list=[]
    list_of_reviews=[]
    base_url =url
    url_list.append(base_url)
    innerPage = requests.get(base_url)
    soup2 = BeautifulSoup(innerPage.content,'html.parser')
    review_pages =math.ceil(int(soup2.find('h2',class_='font20 margin-top10').span.text)/10)
    for i in range(2,review_pages+1):
        url_list.append(base_url+'-p'+str(i))

    for url in url_list:
        page_sub = requests.get(url)
        page_sub_soup = BeautifulSoup(page_sub.content,'html.parser')

        all_reviews =page_sub_soup.find('div',id='userReviewListing').find_all('div')


        for i in range(len(all_reviews)):
            if all_reviews[i].find('a')!=None:
                review_page ='https://www.carwale.com'+all_reviews[i].find('a').get('href')
                list_of_reviews.append(get_review_content(review_page))

    final_df =pd.DataFrame(list_of_reviews)
    return final_df


In [203]:
df_car1 = get_car_reviews('https://www.carwale.com/tata-cars/harrier/userreviews')

In [204]:
df_car1

,Review,comfort,economy,familiarity,like,looks,model,overall,performance,purchase_condition,review_date,total_likes,user_name,value_for_money
0,Good looking and very good performance. This c...,5,4,Have driven for a few hundred kilometres,2,5,XZ,5,5,New,2019-01-25,2,TUSHARKUMAR PARMAR,5
1,Tata Harrier has been the buzzword in the Auto...,5,5,Have driven for a few hundred kilometres,2,5,XE,5,5,New,2019-02-05,2,Devendra Sharma,5
2,The car has a macho look from outside.\nLoade...,4,3,Haven’t driven it,1,5,XE,3,4,Not Purchased,2019-01-27,1,Manish Kedari,3
3,best suv harrier loving it. new launch in tata...,4,3,Have done a short test-drive once,1,4,XZ,3,4,New,2019-01-30,2,Saikumar,4
4,"Design is good ,space is Good but lacking in t...",3,2,Haven’t driven it,1,4,XE,2,3,Not Purchased,2019-01-27,1,Pawandeep,2
5,New tata harrier is awesome and Attractive at ...,5,4,It’s my mate since ages,1,5,XE,4,4,Not Purchased,2019-01-28,1,Sajan,5
6,A Revolutionary Suv car from Tata motors....Be...,5,5,It’s my mate since ages,1,5,XZ,5,5,Not Purchased,2019-01-26,1,Narasimhan,5
7,"Jordar,super look,Lena jaisa,Paisa vasul",5,5,Have driven for a few hundred kilometres,0,5,XE,4,5,New,2019-01-27,0,Hero Ahir,4
8,"It’s perfect SUV with at the best price , it’s...",5,5,Have done a short test-drive once,0,5,XM,5,5,Not Purchased,2019-02-19,0,Abhijeet Kadam,5
9,It is a useful car it is a very smooth car. It...,4,5,Haven’t driven it,0,5,XE,4,5,New,2019-01-24,0,Krishna Birajdar,5


In [205]:
df_car1.shape

(87, 14)

In [212]:
# Create a dataframe and merge with other Dataframe
# import pandas as pd
# dfz = pd.DataFrame(columns=['Car_Name','Review', 'comfort', 'economy', 'familiarity', 'like', 'looks', 'model',
#        'overall', 'performance', 'purchase_condition', 'review_date',
#        'total_likes', 'user_name', 'value_for_money'])
# dfz.head()

# dfy =get_car_reviews('https://www.carwale.com/tata-cars/harrier/userreviews')
# dfy['Car_Name'] = 'Tata Harrier'

# dfz = pd.concat([dfz,dfy])

# dfz.head()


### Create Final DataFrame

In [230]:
def createDataFrame():
    page =requests.get('https://www.carwale.com/tata-cars/')
    soup = BeautifulSoup(page.content,'html.parser')
    test=soup.find('ul',class_='content-box-shadow')
    test2 =test.find_all('li',class_='list-seperator')
    All_df = pd.DataFrame(columns=['Car_Name','Review', 'comfort', 'economy', 'familiarity', 'like', 'looks', 'model',
           'overall', 'performance', 'purchase_condition', 'review_date',
           'total_likes', 'user_name', 'value_for_money'])

    for i in range(len(test2)):
        title = test2[i].find('div',class_='grid-7 omega').a.get('title')
        if title =='Tata 45X':
            continue        
        print(title)
        url ='https://www.carwale.com'+test2[i].find('div',class_='grid-7 omega').a.get('href')+'userreviews'
        df =get_car_reviews(url)
        df['Car_Name'] = title    
        All_df = pd.concat([All_df,df])
        
    return All_df    


In [231]:
final =createDataFrame()

Tata Harrier
Tata Nexon
Tata Tiago
Tata Tigor
Tata Safari Storme
Tata Hexa
Tata Zest
Tata Sumo Gold
Tata Nano GenX
Tata Tiago NRG
Tata Tiago JTP
Tata Bolt
Tata Nano
Tata Tigor JTP


In [233]:
final.shape

(1703, 15)

In [235]:
final.to_csv('Tata Car Reviews.csv',index=False)